In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
from datetime import datetime
from tqdm import tqdm
import time
import jsonlines
import ast
import codecs
import langid

In [2]:
hrefs = []

for i in tqdm(range(471)):
    while True:
        try:
            r = requests.get(f'https://iluminasi.com/?p={i}')
            if r.status_code == 200:
                break
        except:
            time.sleep(1.3)

    soup = BeautifulSoup(r.text, "lxml")
    div = soup.find_all('div', attrs = {"id":"lindex"})

    if len(div) == 0:
        break

    for j in range(len(div[0].find_all("a"))):
        hrefs.append(div[0].find_all("a")[j].get("href"))

100%|████████████████████████████████████████████████████████████████████████████████| 471/471 [04:19<00:00,  1.82it/s]


In [3]:
len(hrefs)

8460

In [4]:
len(set(hrefs))

8460

In [5]:
root_link = "https://iluminasi.com"

In [6]:
for href in tqdm(hrefs):
    
    url = root_link + href
    while True:
        try:
            r = requests.get(url)
            if r.status_code == 200:
                break
        except Exception as e:
            print(e)
            time.sleep(3.9)
            
    soup = BeautifulSoup(r.text, "lxml")
    title = soup.find("h1", attrs = {"itemprop":"name headline"}).text.strip()
    
    div = soup.find_all('div', attrs = {"itemprop":"articleBody"})
    p = [p.text for p in div[0].find_all("p") if len(p.text.split()) > 9]
    
    data = {'url': url, 'headline': title, 'content': p}
    
    with open('iluminasi.jsonl', 'a') as f:
        json.dump(data, f)
        f.write('\n')

100%|████████████████████████████████████████████████████████████████████████████| 8460/8460 [1:24:49<00:00,  1.66it/s]


In [7]:
# Specify the input and output file paths
input_file = "iluminasi.jsonl"
output_file = "iluminasi-cleaned.jsonl"

langid.set_languages(['ms', 'en'])  #ISO 639-1 codes

# Open the input and output files
with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
    # Iterate over each line in the input file
    for line in reader:
        # Check if the content field is empty ([] or {})
        if line['content']:
            # Write the non-empty line to the output file
            decoded_list = []
            for string in line['content']:
                try:
                    decoded_string = codecs.escape_decode(string)[0].decode('utf-8')
                except ValueError:
                    decoded_string = string
                decoded_list.append(decoded_string)
            line['content'] = decoded_list

            writer.write(line)